<a href="https://colab.research.google.com/github/San-Di/scratchML/blob/master/Tensor_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

tf.compat.v1.disable_eager_execution()

In [2]:
epochs = 1
batch_size = 1
learning_rate = 0.01

(X_train,Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 255
X_test = X_test / 255
X_train = X_train.reshape(X_train.shape[0],-1)

img_size_flat = X_train.shape[1]
n_classes = 10

In [3]:
def randomize(x, y):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y.shape[0])
    shuffled_x = x[permutation, :]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y

In [4]:
def get_next_batch(start, end):
  x_batch = X_train[start: end]
  y_batch = Y_train[start: end]
  return x_batch, y_batch
  

In [5]:
def fc(X, neurons, name):

  w_init = tf.compat.v1.truncated_normal_initializer(stddev=0.01)
  W =  tf.compat.v1.get_variable('W' + name,
                           shape=[X.shape[1], neurons],
                           initializer=w_init)


  b_init = tf.constant(0., shape=[1, neurons])
  b = tf.compat.v1.get_variable('b' + name, initializer=b_init)
  print(X.shape, W.shape)
  Z = tf.matmul(X, W)
  A = tf.nn.relu(Z)
  return A

In [6]:
def bp(ground_truth, prediction):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=ground_truth, logits=prediction), name='loss')
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)
  correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1), name='correct_pred')
  accuracy = tf.reduce_mean(tf.cast(correct_prediction), name='accuracy')

  return optimizer, loss, accuracy


In [7]:
def train():
  num_tr_iter = int(len(Y_train) / batch_size)
  init = tf.compat.v1.global_variables_initializer()
  
  x_dict = tf.compat.v1.placeholder(tf.float32, shape=[None, img_size_flat], name='X')
  y_dict = tf.compat.v1.placeholder(tf.float32, shape=[None, n_classes], name='Y')
 
  with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        # print('Training epoch: {}'.format(epoch + 1))
        x_train, y_train = randomize(X_train, Y_train)
        for iteration in range(num_tr_iter):
          start = iteration * batch_size
          end = start + batch_size
          x, y = get_next_batch(start, end)
          
          # print(x.shape, y.shape)
          layer1 = fc(x_dict, 200, str(epoch))
          layer2 = fc(layer1, 10, str(epoch))

          updated_params, loss, acc = bp(y_dict, layer2)

          feed_dict_batch = {x_dict: x, y_dict: y}
          sess.run(updated_params, feed_dict_batch)

          if iteration % 10 == 0:
            # Calculate and display the batch loss and accuracy
            loss_batch, acc_batch = sess.run([loss, acc])

            print("iter {0:3d}:\t Loss={1:.2f},\tTraining Accuracy={2:.01%}".
                  format(iteration, loss_batch, acc_batch))

In [8]:
train()

TypeError: ignored